In [37]:
# 导入需要的模块
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import matplotlib

#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family']='sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [38]:
# 抓取网页
def get_url(url, params=None, proxies=None):
    rsp = requests.get(url, params=params, proxies=proxies)
    rsp.raise_for_status()
    return rsp.text

In [39]:
# 从网页抓取数据
def get_fund_data(code,per=10,sdate='',edate='',proxies=None):
    url = 'http://fund.eastmoney.com/f10/F10DataApi.aspx'
    params = {'type': 'lsjz', 'code': code, 'page':1,'per': per, 'sdate': sdate, 'edate': edate}
    html = get_url(url, params, proxies)
    soup = BeautifulSoup(html, 'html.parser')

    # 获取总页数
    pattern=re.compile(r'pages:(.*),')
    result=re.search(pattern,html).group(1)
    pages=int(result)

    # 获取表头
    heads = []
    for head in soup.findAll("th"):
        heads.append(head.contents[0])

    # 数据存取列表
    records = []

    # 从第1页开始抓取所有页面数据
    page=1
    while page<=pages:
        params = {'type': 'lsjz', 'code': code, 'page':page,'per': per, 'sdate': sdate, 'edate': edate}
        html = get_url(url, params, proxies)
        soup = BeautifulSoup(html, 'html.parser')

        # 获取数据
        for row in soup.findAll("tbody")[0].findAll("tr"):
            row_records = []
            for record in row.findAll('td'):
                val = record.contents

                # 处理空值
                if val == []:
                    row_records.append(np.nan)
                else:
                    row_records.append(val[0])

            # 记录数据
            records.append(row_records)

        # 下一页
        page=page+1

    # 数据整理到dataframe
    np_records = np.array(records)
    data= pd.DataFrame()
    for col,col_name in enumerate(heads):
        data[col_name] = np_records[:,col]

    return data

In [40]:
def readOrDownload(id, startDate='2010-01-01', endDate='2022-07-09'):
#     try:
#         res = readData(id)
#     except:
    res = downloadData(id, sDate= startDate, eDate=endDate)

In [41]:
def readData(id):
    res = pd.read_csv('../data/'+id+'.csv')
    return res

In [42]:
def downloadData(ids, sDate,eDate):
    print("can't read file, now start to download")
    data = get_fund_data(ids, per=49, sdate=sDate, edate=eDate)
    # 修改数据类型
    data['净值日期'] = pd.to_datetime(data['净值日期'], format='%Y/%m/%d')
    data['单位净值'] = data['单位净值'].astype(float)
    data['累计净值'] = data['累计净值'].astype(float)
    data['日增长率'] = data['日增长率'].str.strip('%').astype(float)
    # 按照日期升序排序并重建索引
    data = data.sort_values(by='净值日期', axis=0, ascending=True).reset_index(drop=True)
    saveName = ids + '.csv'
    data.to_csv(saveName, encoding='utf8')
    print('Save successfully!' + saveName)

In [43]:
def selectBy(data, query, column='type'):
    selectData = data.loc[data[column] == query]
    return selectData

In [57]:
data = pd.read_csv('../FundingInfo.csv')
selectedData = selectBy(data, '指数型-股票', column='基金类型')
testdata = np.array( selectedData.values)
a = [1,3]
id_name = testdata[:, a].astype(str)
for i in range(len(id_name[:,0])):
    curID = id_name[i,0]
    while len(curID) < 6:
        curID = '0' + curID
    id_name[i,0] = curID

[['000008' '嘉实中证500ETF联接A']
 ['000042' '财通中证ESG100指数增强A']
 ['000051' '华夏沪深300ETF联接A']
 ...
 ['690008' '民生中证内地资源主题指数A']
 ['700002' '平安深证300指数增强']
 ['740101' '长安沪深300非周期']]


In [58]:
for ids in id_name[:,0]:
    try:
        readOrDownload(ids)
    except:
        continue

can't read file, now start to download
Save successfully!000008.csv
can't read file, now start to download
Save successfully!000042.csv
can't read file, now start to download
Save successfully!000051.csv
can't read file, now start to download
Save successfully!000059.csv
can't read file, now start to download
can't read file, now start to download
can't read file, now start to download
Save successfully!000176.csv
can't read file, now start to download
Save successfully!000248.csv
can't read file, now start to download
Save successfully!000311.csv
can't read file, now start to download
Save successfully!000312.csv
can't read file, now start to download
Save successfully!000313.csv
can't read file, now start to download
Save successfully!000368.csv
can't read file, now start to download
Save successfully!000373.csv
can't read file, now start to download
Save successfully!000376.csv
can't read file, now start to download
Save successfully!000478.csv
can't read file, now start to download

Save successfully!003958.csv
can't read file, now start to download
Save successfully!003986.csv
can't read file, now start to download
Save successfully!004069.csv
can't read file, now start to download
Save successfully!004070.csv
can't read file, now start to download
Save successfully!004190.csv
can't read file, now start to download
Save successfully!004191.csv
can't read file, now start to download
Save successfully!004192.csv
can't read file, now start to download
Save successfully!004193.csv
can't read file, now start to download
Save successfully!004194.csv
can't read file, now start to download
Save successfully!004195.csv
can't read file, now start to download
Save successfully!004342.csv
can't read file, now start to download
Save successfully!004343.csv
can't read file, now start to download
Save successfully!004344.csv
can't read file, now start to download
Save successfully!004345.csv
can't read file, now start to download
Save successfully!004346.csv
can't read file, no

Save successfully!005999.csv
can't read file, now start to download
Save successfully!006020.csv
can't read file, now start to download
Save successfully!006021.csv
can't read file, now start to download
Save successfully!006034.csv
can't read file, now start to download
Save successfully!006048.csv
can't read file, now start to download
Save successfully!006063.csv
can't read file, now start to download
Save successfully!006087.csv
can't read file, now start to download
Save successfully!006098.csv
can't read file, now start to download
Save successfully!006119.csv
can't read file, now start to download
Save successfully!006129.csv
can't read file, now start to download
Save successfully!006130.csv
can't read file, now start to download
Save successfully!006131.csv
can't read file, now start to download
Save successfully!006143.csv
can't read file, now start to download
Save successfully!006144.csv
can't read file, now start to download
Save successfully!006165.csv
can't read file, no

Save successfully!007464.csv
can't read file, now start to download
Save successfully!007465.csv
can't read file, now start to download
Save successfully!007466.csv
can't read file, now start to download
Save successfully!007467.csv
can't read file, now start to download
Save successfully!007472.csv
can't read file, now start to download
Save successfully!007473.csv
can't read file, now start to download
Save successfully!007474.csv
can't read file, now start to download
Save successfully!007475.csv
can't read file, now start to download
Save successfully!007505.csv
can't read file, now start to download
Save successfully!007506.csv
can't read file, now start to download
Save successfully!007531.csv
can't read file, now start to download
Save successfully!007538.csv
can't read file, now start to download
Save successfully!007539.csv
can't read file, now start to download
Save successfully!007541.csv
can't read file, now start to download
Save successfully!007571.csv
can't read file, no

Save successfully!008391.csv
can't read file, now start to download
Save successfully!008396.csv
can't read file, now start to download
Save successfully!008397.csv
can't read file, now start to download
Save successfully!008399.csv
can't read file, now start to download
Save successfully!008400.csv
can't read file, now start to download
Save successfully!008407.csv
can't read file, now start to download
Save successfully!008408.csv
can't read file, now start to download
Save successfully!008519.csv
can't read file, now start to download
Save successfully!008520.csv
can't read file, now start to download
Save successfully!008551.csv
can't read file, now start to download
Save successfully!008552.csv
can't read file, now start to download
Save successfully!008585.csv
can't read file, now start to download
Save successfully!008586.csv
can't read file, now start to download
Save successfully!008590.csv
can't read file, now start to download
Save successfully!008591.csv
can't read file, no

...

Save successfully!180003.csv
can't read file, now start to download
Save successfully!200002.csv
can't read file, now start to download
can't read file, now start to download
Save successfully!202015.csv
can't read file, now start to download
can't read file, now start to download
Save successfully!202017.csv
can't read file, now start to download
can't read file, now start to download
Save successfully!202021.csv
can't read file, now start to download
can't read file, now start to download
Save successfully!202025.csv
can't read file, now start to download
can't read file, now start to download
Save successfully!202211.csv
can't read file, now start to download
Save successfully!213010.csv
can't read file, now start to download
Save successfully!217016.csv
can't read file, now start to download
Save successfully!217017.csv
can't read file, now start to download
Save successfully!217019.csv
can't read file, now start to download
Save successfully!217027.csv
can't read file, now start t

Save successfully!510410.csv
can't read file, now start to download
Save successfully!510440.csv
can't read file, now start to download
Save successfully!510500.csv
can't read file, now start to download
Save successfully!510510.csv
can't read file, now start to download
Save successfully!510530.csv
can't read file, now start to download
Save successfully!510550.csv
can't read file, now start to download
Save successfully!510560.csv
can't read file, now start to download
Save successfully!510570.csv
can't read file, now start to download
Save successfully!510580.csv
can't read file, now start to download
Save successfully!510590.csv
can't read file, now start to download
Save successfully!510600.csv
can't read file, now start to download
Save successfully!510630.csv
can't read file, now start to download
Save successfully!510650.csv
can't read file, now start to download
Save successfully!510660.csv
can't read file, now start to download
Save successfully!510680.csv
can't read file, no

Save successfully!515160.csv
can't read file, now start to download
Save successfully!515170.csv
can't read file, now start to download
Save successfully!515180.csv
can't read file, now start to download
Save successfully!515190.csv
can't read file, now start to download
Save successfully!515200.csv
can't read file, now start to download
Save successfully!515210.csv
can't read file, now start to download
Save successfully!515220.csv
can't read file, now start to download
Save successfully!515230.csv
can't read file, now start to download
Save successfully!515250.csv
can't read file, now start to download
Save successfully!515260.csv
can't read file, now start to download
Save successfully!515280.csv
can't read file, now start to download
Save successfully!515290.csv
can't read file, now start to download
can't read file, now start to download
Save successfully!515300.csv
can't read file, now start to download
Save successfully!515310.csv
can't read file, now start to download
Save succ

Save successfully!516800.csv
can't read file, now start to download
Save successfully!516810.csv
can't read file, now start to download
Save successfully!516820.csv
can't read file, now start to download
Save successfully!516830.csv
can't read file, now start to download
Save successfully!516850.csv
can't read file, now start to download
Save successfully!516860.csv
can't read file, now start to download
Save successfully!516880.csv
can't read file, now start to download
Save successfully!516890.csv
can't read file, now start to download
Save successfully!516900.csv
can't read file, now start to download
Save successfully!516910.csv
can't read file, now start to download
Save successfully!516920.csv
can't read file, now start to download
Save successfully!516930.csv
can't read file, now start to download
Save successfully!516950.csv
can't read file, now start to download
Save successfully!516960.csv
can't read file, now start to download
Save successfully!516970.csv
can't read file, no

Save successfully!588280.csv
can't read file, now start to download
Save successfully!588300.csv
can't read file, now start to download
Save successfully!588310.csv
can't read file, now start to download
Save successfully!588330.csv
can't read file, now start to download
Save successfully!588360.csv
can't read file, now start to download
Save successfully!588380.csv
can't read file, now start to download
Save successfully!588390.csv
can't read file, now start to download
Save successfully!588400.csv
can't read file, now start to download
Save successfully!590007.csv
can't read file, now start to download
Save successfully!660008.csv
can't read file, now start to download
Save successfully!660011.csv
can't read file, now start to download
Save successfully!673100.csv
can't read file, now start to download
Save successfully!673101.csv
can't read file, now start to download
Save successfully!690008.csv
can't read file, now start to download
Save successfully!700002.csv
can't read file, no

In [31]:
ids = str(ids)
ids

'159609'

In [32]:
readOrDownload(ids)

can't read file, now start to download
